In [1]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

C:\Users\ishan\anaconda3\envs\quantVE\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
data = yf.download("^GSPC", period="2y", interval="1d")
data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-06-17,2889.750000,2897.270020,2887.300049,2889.669922,2889.669922,2810140000
2019-06-18,2906.709961,2930.790039,2905.439941,2917.750000,2917.750000,3437620000
2019-06-19,2920.550049,2931.739990,2911.429932,2926.459961,2926.459961,3287890000
2019-06-20,2949.600098,2958.060059,2931.500000,2954.179932,2954.179932,3905940000
2019-06-21,2952.709961,2964.149902,2946.870117,2950.459961,2950.459961,5000120000


In [3]:
def EMA(values, n):
    return pd.Series(values).ewm(span=n,min_periods=n).mean()

In [4]:
class GoldenCrossover(Strategy):
    def init(self):
        price = self.data['Adj Close']
        self.ma1 = self.I(EMA, price, 20)
        self.ma2 = self.I(EMA, price, 50)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.position.close()
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.position.close()
            self.sell()

In [5]:
bt = Backtest(data, GoldenCrossover, commission=.002,
              exclusive_orders=True, cash=10000)


In [6]:
stats = bt.run()
stats

Start                     2019-06-17 00:00:00
End                       2021-06-15 00:00:00
Duration                    729 days 00:00:00
Exposure Time [%]                     87.8968
Equity Final [$]                      13386.9
Equity Peak [$]                       13386.9
Return [%]                            33.8686
Buy & Hold Return [%]                 46.8995
Return (Ann.) [%]                     15.7016
Volatility (Ann.) [%]                 24.8398
Sharpe Ratio                         0.632114
Sortino Ratio                         1.07922
Calmar Ratio                         0.731397
Max. Drawdown [%]                     -21.468
Avg. Drawdown [%]                    -2.28274
Max. Drawdown Duration      253 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    5
Win Rate [%]                               40
Best Trade [%]                        44.4731
Worst Trade [%]                      -3.44925
Avg. Trade [%]                    

In [8]:
trade=stats['_trades']
trade

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,3,61,80,3015.098238,2911.100098,-311.994422,-0.034492,2019-09-12,2019-10-09,27 days
1,-3,80,85,2905.277897,2989.679932,-253.206103,-0.029051,2019-10-09,2019-10-16,7 days
2,3,85,178,2995.659292,2974.280029,-64.137787,-0.007137,2019-10-16,2020-03-02,138 days
3,-3,178,228,2968.331469,2939.500000,86.494408,0.009713,2020-03-02,2020-05-12,71 days
4,3,228,503,2945.379000,4255.279785,3929.702355,0.444731,2020-05-12,2021-06-15,399 days


In [9]:
bt.plot()